In [199]:
import pandas as pd
import pyfredapi as pf
import requests
import os
import statsmodels.api as sm
import re


#graph.viz
#imputation to fill the gaps in the data

**Converts Data Into Quarter over Quarter Percent Change**
<pre>
- goes through the dataframe column by column
- will use the pct_change method to calculate the quarter by quarter percent change (because each row is the start of a new quarter)
- drops all the empty rows (first one because it has no previous data to compare to)
- will reset the index to make up for losing the first row

In [162]:
#loops over dataframe columns does the QoQ % change for everything except the date
def convert_QoQ_change(data):
    count = 0
    for col in data.columns:
    # compute pct_change on the Series, then assign back into df
        if count != 0:
            data[col] = data[col].pct_change(fill_method=None)*100
        count = count + 1

    data = data.dropna()
    data = data.reset_index(drop=True)
    return data

**Converts CSV data that has a DAILY frequency into a proper dataframe with filled data**
<pre>
- This will first convert it into a dataframe
- Next it will use interpolation to fill in the missing data (will work only if very minimal data is missing, max a couple days at a time every once in a while)
- fixes up the dataframe and resets its index
- returns fixed up dataframe now in quarter to quarter % change

In [154]:
def daily_conversion(data, date_name):
    
    #converts the string date into a datetime object
    data[date_name] = pd.to_datetime(data[date_name])
    
    data = data.set_index(date_name).sort_index()
    data = data.asfreq('D')#fills in every calendar day thats missing, and places NaaN for each value it filled
    
    if pd.isna(data).any().any():
        data = data.interpolate(method='time')
    
    data = data.asfreq('QS')
    data = data.reset_index()
    
    data = convert_QoQ_change(data)
    
    return data

**This would deal with yearly data**
<pre>
- This would deal with datasets that are on a yearly NUMERICAL bases
- it would forward fill the data in order to fit for a quarter to quarter change
    - Concerns (approach is subject to change at the moment)
        - Will most likely to create bias/disrupt the data when calculating edge weights
- As you can see from the sample below it doesn't seem to be very affective

In [187]:
def yearly_restructure(data, date_name,numerical = True):
    
    
    data[date_name] = pd.to_datetime(data[date_name], format='%Y')
    data = data.set_index(date_name).sort_index()
    
    data = data.loc['1999':"2024"]#restricts data within time frame
    
    data = data.asfreq('QS')#fills in every quarter thats missing, and places NaaN for each value it filled
    data = data.ffill()#Forwards fills the NaaN values
    
    data = data.reset_index()
    
    if numerical == True:
        data = convert_QoQ_change(data)
        
    
    data = data[data['Year'] >= '2000-01-01']
    
    
    return data
    
    

military_spending = pd.read_csv('data/United-States-Military-SpendingDefense-Budget-Military-Spending.csv')
date_ms = military_spending.columns[0]


military_spending = yearly_restructure(military_spending, date_ms)
military_spending
    

,Year,Military Spending/Defense Budget
3,2000-01-01,7
4,2000-04-01,0
5,2000-07-01,0
6,2000-10-01,0
7,2001-01-01,4
...,...,...
95,2023-01-01,6
96,2023-04-01,0
97,2023-07-01,0
98,2023-10-01,0


**Converts year long gapped data into a properly strucutred quarter to quarter filled data**

<pre>
- This would mainly be for things like majority party in congress, or party in presidency, or bills/rules that change every so couple of years
- Date column needed to be restrucutred
    - Originally in Exp: 115(Congress) (2003-2005)
    - Converter holds a needed year only

Approach:
- Just concentrate on the starting year, then depend on forward fill within the method in order to adjust the rest of the data

In [232]:
government_ideology = pd.read_csv('Data/Government ideology Datasheet.csv')
gov_ideology_date = government_ideology.columns[0]

#restructures the year data
for i in range(len(government_ideology)):
    row = government_ideology.iloc[i]
    row = row.iloc[0]

    # Split using en dash, not a regular dash
    desired_part = row.split('(')[1].split('–')[0]  # "KeepThisPart"

    government_ideology.loc[i, gov_ideology_date] = desired_part

#restructures the presidency data to keep only the party, since the name of the president isnt relevant
government_ideology['Presidency'] = (
    government_ideology['Presidency']
      # 1) drop any “ (…)" suffix
      .str.replace(r'\s*\(.*\)', '', regex=True)
      # 2) remove stray digits
      .str.replace(r'\d+',      '', regex=True)
      # 3) trim any extra whitespace
      .str.strip()
)

government_ideology = yearly_restructure(government_ideology, gov_ideology_date, numerical=False)
government_ideology

,Year,House Majority,Senate Majority,Presidency,Party Government
4,2000-01-01,Republicans,Republicans,Democrat,Divided
5,2000-04-01,Republicans,Republicans,Democrat,Divided
6,2000-07-01,Republicans,Republicans,Democrat,Divided
7,2000-10-01,Republicans,Republicans,Democrat,Divided
8,2001-01-01,Republicans,Republicans / Democrats12,Republican,Unified / Divided
...,...,...,...,...,...
92,2022-01-01,Democrats,Democrats14,Democrat,Unified
93,2022-04-01,Democrats,Democrats14,Democrat,Unified
94,2022-07-01,Democrats,Democrats14,Democrat,Unified
95,2022-10-01,Democrats,Democrats14,Democrat,Unified


**Pulls Data From Fred API**

In [110]:

#adds all of the variables into a temperary dataframe using the pyfredapi library
sc = pf.SeriesCollection(
    series_id=["GDP"]
)

sc.add(series_id=["FEDFUNDS"])
sc.add(series_id=["MTSDS133FMS"])
sc.add(series_id=["FYGFDPUN"])
sc.add(series_id=["FGRECPT"])
sc.add(series_id=["FYONGDA188S"])
sc.add(series_id=["UNRATE"])
sc.add(series_id=["SPPOP65UPTOZSUSA"])
sc.add(series_id=["CPIAUCSL"])
sc.add(series_id=["GFDEBTN"])
sc.add(series_id=["GDP"])
sc.add(series_id=["EFFR"])
sc.add(series_id=["DGS10"])

#Renaming all of the variables
new_names = {
    "GDP": "GDP",
    "FEDFUNDS": "Federal Funds Effective Rate",
    "MTSDS133FMS": "Federal Surplus or Deficit",
    "FYGFDPUN": "Federal Debt Held by the Public" ,
    "FGRECPT": "Federal Revenue",
    "NA000283Q":"Federal Spending" ,
    "UNRATE": "Unemployment rate",
    "W823RC1": "Using Social Security",
    "CPIAUCSL": "Inflation rate", 
    "GFDEBTN": "National Debt"
    
}
#"DGS10": "Treasury Yields"
#,"EFFR": "Federal Funds Rate"

#



'''
Variables to look into adding: Federal government budget surplus or deficit (-) (M318501Q027NBEA)
Federal government current tax receipts(how much the government collected in taxes) (W006RC1Q027SBEA)
- War
- Ideaoligy
    - Look into how to work with senate and house of rep data again along with adding presidency
    - For congress(https://history.house.gov/Institution/Party-Divisions/Party-Divisions/)
    - for Presidency()
- The feds monetary policy
-Government spending allocation
    - Maybe finding some sort of dataset that has each govenemnt spending spread from year to year
    - military Budget (https://www.macrotrends.net/global-metrics/countries/usa/united-states/military-spending-defense-budget)
'''



sc = pf.SeriesCollection(series_id=["GDP","FEDFUNDS","MTSDS133FMS","FYGFDPUN","FGRECPT","NA000283Q","UNRATE","W823RC1","CPIAUCSL","GFDEBTN"], rename=new_names)

#converts it into a dataframe where each variable has its own column
cpi_long = sc.merge_wide()

#converts from temperary dataframe being returned into a real dataframe
df = pd.DataFrame(cpi_long)

#removes unnecessary dates(keeps 1999-2024) and reorganizes index

df["date"]= pd.to_datetime(df["date"])
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

df = df.loc['1999-07-01':"2024-01-01"]#limits the dataframe to this time constraint
df = df.dropna()#removes unnecessary lines
df = convert_QoQ_change(df)


Requesting series GDP...
Requesting series FEDFUNDS...
Requesting series MTSDS133FMS...
Requesting series FYGFDPUN...
Requesting series FGRECPT...
Requesting series FYONGDA188S...
Requesting series UNRATE...
Requesting series SPPOP65UPTOZSUSA...
Requesting series CPIAUCSL...
Requesting series GFDEBTN...
Requesting series GDP...
Requesting series EFFR...
Requesting series DGS10...
Requesting series GDP...
Requesting series FEDFUNDS...
Requesting series MTSDS133FMS...
Requesting series FYGFDPUN...
Requesting series FGRECPT...
Requesting series NA000283Q...
Requesting series UNRATE...
Requesting series W823RC1...
Requesting series CPIAUCSL...
Requesting series GFDEBTN...


**This section will begin to calculate the edge weights for the variables**

**National Debt**
<pre>
Edges and their weights:

GDP: 32.0917
Federal Surplus or deficit:
Federal Spending: 

In [54]:
independent = ["GDP","Federal Surplus or Deficit", "Federal Spending"]
dependent = "National Debt"


X = df[independent]          # DataFrame of just the parent nodes
X = sm.add_constant(X)   # adds intercept column
y = df[dependent]       # Series of your target


model = sm.OLS(y, X).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:          National Debt   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     12.24
Date:                Thu, 03 Jul 2025   Prob (F-statistic):           8.69e-07
Time:                        16:11:28   Log-Likelihood:                -174.30
No. Observations:                  94   AIC:                             356.6
Df Residuals:                      90   BIC:                             366.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

**Federal Surplus or Deficit**
<pre>
Edges and their weights:
Federal Debt Held by the Public: 32.0917

In [260]:
independent_1 = ["Federal Debt Held by the Public"]
dependent_1 = "Federal Surplus or Deficit"


X_1 = df[independent_1]          # DataFrame of just the parent nodes
X_1 = sm.add_constant(X_1)   # adds intercept column
y_1 = df[dependent_1]       # Series of your target


model_1 = sm.OLS(y_1, X_1).fit()
print(model_1.summary())


                                OLS Regression Results                                
Dep. Variable:     Federal Surplus or Deficit   R-squared:                       0.026
Model:                                    OLS   Adj. R-squared:                  0.016
Method:                         Least Squares   F-statistic:                     2.512
Date:                        Thu, 03 Jul 2025   Prob (F-statistic):              0.116
Time:                                00:02:07   Log-Likelihood:                -752.23
No. Observations:                          97   AIC:                             1508.
Df Residuals:                              95   BIC:                             1514.
Df Model:                                   1                                         
Covariance Type:                    nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

**Edge Weights to Federal Spendings**
<pre>
Edges and their weights:
Federal Debt Held by the Public: 1.6685
Using Social Security: -0.1599

In [104]:
independent_2 = ["Federal Debt Held by the Public","Using Social Security"]
dependent_2 = "Federal Spending"


X_2 = df[independent_2]          # DataFrame of just the parent nodes
X_2 = sm.add_constant(X_2)   # adds intercept column
y_2 = df[dependent_2]       # Series of your target


model_2 = sm.OLS(y_2, X_2).fit()
print(model_2.summary())


                            OLS Regression Results                            
Dep. Variable:       Federal Spending   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     13.84
Date:                Tue, 08 Jul 2025   Prob (F-statistic):           5.32e-06
Time:                        17:56:16   Log-Likelihood:                -351.82
No. Observations:                  98   AIC:                             709.6
Df Residuals:                      95   BIC:                             717.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

**Series of Presidents, House of Representatives and Senate Control**
<pre>
How the parties will be represented in the Data

0 indicates Democrat (Blue)
1 Represents Republican (Red)

Data pulled from: 
https://api.congress.gov

Attempting to do an OOP version of the previous pull of info

In [115]:
treasury_Yield_oop = pd.read_csv("Data/Treasury_Yields.csv")
date_section_name = 'observation_date'

treasury_Yield_oop = daily_conversion(treasury_Yield_oop, date_section_name)
treasury_Yield_oop

,observation_date,DGS10(Treasury_Yield)
0,1999-10-01,2.564103
1,2000-01-01,8.222222
2,2000-04-01,-7.289528
3,2000-07-01,0.000000
4,2000-10-01,-3.322259
...,...,...
93,2023-01-01,1.544572
94,2023-04-01,-9.691439
95,2023-07-01,10.490857
96,2023-10-01,21.689895
